<a href="https://colab.research.google.com/github/goransavich/cnn-xgboost/blob/main/cnn-xgboost-final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot
np.random.seed(1337)  # for reproducibility
#from keras.utils import np_utils
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
#xgboost
import xgboost as xgb
import pandas as pd
from sklearn import metrics

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd

train_name = list(uploaded.keys())[0]

train_df = pd.read_csv(io.BytesIO(uploaded[train_name]))

Saving Meat_TRAIN.csv to Meat_TRAIN.csv


In [ ]:
uploaded = files.upload()

test_name = list(uploaded.keys())[0]

test_df = pd.read_csv(io.BytesIO(uploaded[test_name]))

Saving Meat_TEST.csv to Meat_TEST.csv


In [ ]:
train_data = np.array(train_df)
test_data = np.array(test_df)

In [ ]:
x_train = train_data[:, :-1]
x_test = test_data[:, :-1]
y_train = train_data[:, -1:]
y_test = test_data[:, -1:]

classes_in_dataset = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

# transform the labels from integers to one hot vectors
enc = OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

In [ ]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)

In [ ]:
if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
input_shape = x_train.shape[1:]

print(input_shape)


print(classes_in_dataset)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validate.shape)
print(y_validate.shape)

(448, 1)
3
(48, 448, 1)
(48, 3)
(60, 448, 1)
(60, 3)
(12, 448)
(12, 3)


In [ ]:
inputs = input_shape
input_layer = Input(inputs)
conv2d1 = Convolution1D(filters=64, kernel_size=1, padding='valid')(input_layer)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation1)
#dropout1 = Dropout(0.5)(maxpooling1)

conv2d2 = Convolution1D(filters=62, kernel_size=1, padding='valid')(maxpooling1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation2)
dropout2 = Dropout(0.5)(maxpooling2)

flatten = Flatten()(maxpooling2)

output_for_xgboost = Model(inputs=input_layer, outputs=flatten)

######### XGBOOST #############
x_train_xg = output_for_xgboost.predict(x_train)
x_test_xg = output_for_xgboost.predict(x_test)

train_x = x_train_xg
test_x = x_test_xg
#print(x_train_xg.shape)
#print(x_test_xg.shape)

dataset = xgb.DMatrix(train_x, label=y_train)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class': classes_in_dataset,'objective':'multi:softmax' } 
model_xg = xgb.train(params, dataset, num_boost_round=150, evals=watchlist)


test_x = xgb.DMatrix(test_x)
result = model_xg.predict(test_x)


In [ ]:

from sklearn.metrics import precision_score, recall_score, accuracy_score

#preds = model.predict(d_test)
best_preds = np.asarray([np.argmax(line) for line in result])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))